In [1]:
!pip install sgp4

   ---------------------------------------- 0.0/160.8 kB ? eta -:--:--
   -------------------------------------- - 153.6/160.8 kB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 160.8/160.8 kB 3.3 MB/s eta 0:00:00


In [7]:
!pip install skyfield

   ---------------------------------------- 0.0/336.2 kB ? eta -:--:--
   ----------------------------- ---------- 245.8/336.2 kB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 336.2/336.2 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/47.2 kB ? eta -:--:--
   ---------------------------------------- 47.2/47.2 kB 2.3 MB/s eta 0:00:00


In [3]:
from sgp4.api import Satrec
from sgp4.api import WGS72
import numpy as np

In [15]:
line1 = '1 28654U 05018A   23179.75299769  .00000097  00000+0  85403-4 0  9995'
line2 = '2 28654  99.1427 163.1612 0012535  38.2684  16.2937 14.12420431933700'

satellite = Satrec.twoline2rv(line1, line2)

In [16]:
def propagate_satellite(satellite, jd, fr):
    e, r, v = satellite.sgp4(jd, fr)
    return r

In [18]:
minutes_per_day = 1440
jd_start = 23300  # Example Julian date for start of propagation

# Arrays to store latitude and longitude
latitudes = []
longitudes = []

# Propagate satellite position over one day at 1-minute intervals
for minute in range(minutes_per_day):
    jd = jd_start + minute / minutes_per_day
    fr = minute / minutes_per_day - int(minute / minutes_per_day)
    r = propagate_satellite(satellite, jd, fr)
    
    # Convert position vector to latitude and longitude
    x, y, z = r
    norm = np.linalg.norm([x, y, z])
    lat = np.arcsin(z / norm) * 180 / np.pi
    lon = np.arctan2(y, x) * 180 / np.pi

    latitudes.append(lat)
    longitudes.append(lon)

# Find the minimum and maximum latitudes and longitudes
min_lat = min(latitudes)
max_lat = max(latitudes)
min_lon = min(longitudes)
max_lon = max(longitudes)

print(f"Minimum Latitude: {min_lat}")
print(f"Maximum Latitude: {max_lat}")
print(f"Minimum Longitude: {min_lon}")
print(f"Maximum Longitude: {max_lon}")

Minimum Latitude: -80.856735803025
Maximum Latitude: 80.85631408575544
Minimum Longitude: -178.90587817703394
Maximum Longitude: 176.68730920189313


In [26]:
from skyfield.api import load, EarthSatellite
import numpy as np

# Load TLE data
tle_lines = [
    '1 28654U 05018A   23179.75299769  .00000097  00000+0  85403-4 0  9995',
    '2 28654  99.1427 163.1612 0012535  38.2684  16.2937 14.12420431933700'
]
satellite = EarthSatellite(tle_lines[0], tle_lines[1], 'NOAA 18', load.timescale())

# Time range for the analysis
ts = load.timescale()
times = ts.utc(2023, 10, 27,10,40,00) # year,month,day,hour,min,sec

# Calculate satellite positions
geocentric = satellite.at(times)

# Convert to geographic coordinates
subpoint = geocentric.subpoint()
latitudes = subpoint.latitude.degrees
longitudes = subpoint.longitude.degrees

# Find max and min latitude and longitude
max_latitude = np.max(latitudes)
min_latitude = np.min(latitudes)
max_longitude = np.max(longitudes)
min_longitude = np.min(longitudes)

print(f"Max Latitude: {max_latitude} degrees")
print(f"Min Latitude: {min_latitude} degrees")
print(f"Max Longitude: {max_longitude} degrees")
print(f"Min Longitude: {min_longitude} degrees")


Max Latitude: -34.488968353457146 degrees
Min Latitude: -34.488968353457146 degrees
Max Longitude: 97.01752829895618 degrees
Min Longitude: 97.01752829895618 degrees
